In [1]:
import pandas as pd
from sqlalchemy import create_engine


In [2]:
!pip install pandas sqlalchemy psycopg2-binary


In [ ]:
POSTGRES_USER = "nishantsingh"
POSTGRES_PASS = "MViOfwCs2ZrSVoV3"
POSTGRES_HOST = "data-analytics-course-2.c8g8r1deus2v.eu-central-1.rds.amazonaws.com"
POSTGRES_PORT = "5432"
POSTGRES_DB = "jalapeno_joins"
POSTGRES_SCHEMA = "s_nishantsingh"


In [4]:
DB_URL = (
    "postgresql+psycopg2://"
    "nishantsingh:MViOfwCs2ZrSVoV3"
    "@data-analytics-course-2.c8g8r1deus2v.eu-central-1.rds.amazonaws.com:5432"
    "/jalapeno_joins"
)


In [5]:
from sqlalchemy import create_engine

DB_URL = (
    "postgresql+psycopg2://"
    "nishantsingh:MViOfwCs2ZrSVoV3"
    "@data-analytics-course-2.c8g8r1deus2v.eu-central-1.rds.amazonaws.com:5432"
    "/jalapeno_joins"
)

engine = create_engine(DB_URL)
engine.connect()


In [6]:
import os
os.chdir(r"c:\Users\nisha\marketing_and_sales_funnel_analysis")
os.getcwd()


'c:\\Users\\nisha\\marketing_and_sales_funnel_analysis'

In [7]:
os.listdir("data_raw")


['olist_closed_deals_dataset.csv',
 'olist_customers_dataset.csv',
 'olist_geolocation_dataset.csv',
 'olist_marketing_qualified_leads_dataset.csv',
 'olist_orders_dataset.csv',
 'olist_order_items_dataset.csv',
 'olist_order_payments_dataset.csv',
 'olist_order_reviews_dataset.csv',
 'olist_products_dataset.csv',
 'olist_sellers_dataset.csv',
 'product_category_name_translation.csv']

In [8]:
import pandas as pd
import os

# Folder and files
folder = "data_raw"
files = [f for f in os.listdir(folder) if f.endswith(".csv")]

# Function to detect date-like columns
def find_date_columns(filename):
    df = pd.read_csv(os.path.join(folder, filename), nrows=100)  # sample only
    date_cols = []
    for col in df.columns:
        if "date" in col.lower() or "time" in col.lower() or "timestamp" in col.lower():
            date_cols.append(col)
        elif pd.to_datetime(df[col], errors="coerce").notna().sum() > 0.8 * len(df):
            date_cols.append(col)
    return date_cols

# Scan all files
date_columns_map = {}
for file in files:
    date_columns_map[file] = find_date_columns(file)

# Show results
for file, cols in date_columns_map.items():
    print(f"{file}: {cols}")


C:\Users\nisha\AppData\Local\Temp\ipykernel_33504\3807102194.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  elif pd.to_datetime(df[col], errors="coerce").notna().sum() > 0.8 * len(df):
C:\Users\nisha\AppData\Local\Temp\ipykernel_33504\3807102194.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  elif pd.to_datetime(df[col], errors="coerce").notna().sum() > 0.8 * len(df):
C:\Users\nisha\AppData\Local\Temp\ipykernel_33504\3807102194.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  elif pd.to_datetime(df[col], errors="coerce").notna().sum() > 0.8 * len(df):
C:\User

olist_closed_deals_dataset.csv: ['won_date', 'declared_monthly_revenue']
olist_customers_dataset.csv: ['customer_zip_code_prefix']
olist_geolocation_dataset.csv: ['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']
olist_marketing_qualified_leads_dataset.csv: ['first_contact_date']
olist_orders_dataset.csv: ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
olist_order_items_dataset.csv: ['order_item_id', 'shipping_limit_date', 'price', 'freight_value']
olist_order_payments_dataset.csv: ['payment_sequential', 'payment_installments', 'payment_value']
olist_order_reviews_dataset.csv: ['review_score', 'review_creation_date', 'review_answer_timestamp']
olist_products_dataset.csv: ['product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']
olist_sellers_dataset.csv: ['seller_zip_cod

C:\Users\nisha\AppData\Local\Temp\ipykernel_33504\3807102194.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  elif pd.to_datetime(df[col], errors="coerce").notna().sum() > 0.8 * len(df):
C:\Users\nisha\AppData\Local\Temp\ipykernel_33504\3807102194.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  elif pd.to_datetime(df[col], errors="coerce").notna().sum() > 0.8 * len(df):
C:\Users\nisha\AppData\Local\Temp\ipykernel_33504\3807102194.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  elif pd.to_datetime(df[col], errors="coerce").notna().sum() > 0.8 * len(df):
C:\User

In [9]:
parse_dates = {
    "olist_orders_dataset.csv": [
        "order_purchase_timestamp",
        "order_approved_at",
        "order_delivered_carrier_date",
        "order_delivered_customer_date",
        "order_estimated_delivery_date"
    ],
    "olist_order_items_dataset.csv": [
        "shipping_limit_date"
    ],
    "olist_order_reviews_dataset.csv": [
        "review_creation_date",
        "review_answer_timestamp"
    ],
    "olist_marketing_qualified_leads_dataset.csv": [
        "first_contact_date"
    ],
    "olist_closed_deals_dataset.csv": [
        "won_date"
    ]
}


In [10]:
def load_csv_to_postgres(filename, table_name):
    file_path = f"data_raw/{filename}"
    dates = parse_dates.get(filename, None)

    print(f"\nLoading {filename} into {table_name}...")

    df = pd.read_csv(file_path, parse_dates=dates)

    # Visibility section
    print(f"Shape: {df.shape}")
    print("\nData types:")
    print(df.dtypes)
    print("\nSample rows:")
    print(df.head(2))

    df.to_sql(
        table_name,
        engine,
        schema="s_nishantsingh",
        if_exists="replace",
        index=False
    )

    print(f"Loaded {table_name} successfully.\n")


In [ ]:
file_table_map = {
    "olist_orders_dataset.csv": "orders",
    "olist_order_items_dataset.csv": "order_items",
    "olist_order_payments_dataset.csv": "order_payments",
    "olist_order_reviews_dataset.csv": "order_reviews",
    "olist_products_dataset.csv": "products",
    "olist_customers_dataset.csv": "customers",
    "olist_sellers_dataset.csv": "sellers",
    "olist_geolocation_dataset.csv": "geolocation",
    "product_category_name_translation.csv": "product_category_name_translation",
    "olist_marketing_qualified_leads_dataset.csv": "mql",
    "olist_closed_deals_dataset.csv": "closed_deals"
}
for filename, table_name in file_table_map.items():
    load_csv_to_postgres(filename, table_name)
